# EddyPro API

This notebook gives an outline for working with the API

## Create Optional Auxillary Input Data

* It's best practice to give EddyPro Biomet data (temperature, radiation, etc.) and any dynamic metadata (canopy height) in a .csv file.  Biomet.Net users can create the files from database traces using the code below.  You can also create these files yourself

    * Biomet files: https://www.licor.com/env/support/EddyPro/topics/biomet-data-format.html
    * Dynamic metadata files: https://www.licor.com/env/support/EddyPro/topics/dynamic-metadata.html

In [146]:
# Create biomet and dynamicMetadata.csv files
import os
import sys
import importlib

BiometNet = 'C:/Biomet.net/Python/'

# UBC Micromet users can 
if sys.path[0]!=BiometNet:
    sys.path.insert(0,BiometNet)

wd = [p for p in sys.path if p != BiometNet][0]

import csvFromBinary as cfb
importlib.reload(cfb)

siteID = 'BBS'
dateRange = ['2023-06-01','2024-05-31']
tasks=os.path.abspath(wd+'/config_files/EP_auxillary_data_defs.yml')
auxilaryDpaths=cfb.makeCSV(siteID,dateRange,tasks,
                           outputPath=f'C:/highfreq/{siteID}/auxilaryData',stage='Second')
print(auxilaryDpaths)

Initializing tasks for BBS over: ['2023-06-01', '2024-05-31']
PPFD_IN_1_1_1 missing, outputting NaNs
See output: C:/highfreq/BBS/auxilaryData/BBS_biometData_202306010000_202405310000.csv
See output: C:/highfreq/BBS/auxilaryData/BBS_dynamicMetadata_202306010000_202405310000.csv
All tasks completed successfully
{'biometData': 'C:/highfreq/BBS/auxilaryData/BBS_biometData_202306010000_202405310000.csv', 'dynamicMetadata': 'C:/highfreq/BBS/auxilaryData/BBS_dynamicMetadata_202306010000_202405310000.csv'}


In [2]:
L = ['group_1_a','group_1_b','group_2_a']
for groupID in ['1','2']:
    ix = [g for g in L if f"group_{groupID}" in g]
    print(ix)

['group_1_a', 'group_1_b']
['group_2_a']


In [6]:
import pandas as pd

from datetime import datetime,date

r = pd.DatetimeIndex([date(datetime.now().year,1,1),datetime.now()])
print(f"{r.strftime(date_format='%Y-%m-%d %H:%M').values}")


['2024-01-01 00:00' '2024-06-20 11:19']


In [161]:
import eddyProAPI
import pandas as pd
import importlib
import time
importlib.reload(eddyProAPI)

# Basic call for .ghg files

siteID = 'BBS'
dateRange=[(pd.Timestamp.now()-pd.Timedelta(days=365)).strftime('%Y%m%d'),pd.Timestamp.now().strftime('%Y%m%d')],


if siteID == 'BB':
    pp = eddyProAPI.preProcessing('BB',dateRange=dateRange,Testing=5)
    pp.searchRawDir()
elif siteID == 'BBS':
                
    # # Basic call for .dat files
    metaDataTemplate='C:/highfreq/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata'
    pp = eddyProAPI.preProcessing('BBS',dateRange=dateRange,fileType='dat',metaDataTemplate=metaDataTemplate,
                                # debug=True,
                                # reset=True,
                                # testSet=15
    )
#     pp.searchRawDir(
#                 copyFrom = "C:/Campbellsci/CardConvert",
#                 searchTag = "BBS.FLUX",
#                 timeShift=30)
# pp.readFiles()
pp.groupAndFilter()
# with shutil 186.0170259475708 seconds
# via subprocess

In [162]:
import pandas as pd
# Cospectral correction options
hf_meth = {
    'Moncrieff et al. (1997)':1,
    'Horst (1997)':2,
    'Ibrom et al. (2007)':3,
    'Fratini et al. (2012)':4, # Recommended , when sufficient data (>1 month) are available
    'Massman (2000, 2001)':5
}

userDefinedEddyProSettings = {
    'Project':{
        'hf_meth':'1'
        # 'hf_meth':hf_meth['Fratini et al. (2012)'],
        },
    'RawProcess_Settings':{
        'v_offset':0.08,
        'u_offset':-0.04
        },
    }

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)

siteID = 'BBS'

dateRange=[(pd.Timestamp.now()-pd.Timedelta(days=60)).round('30T').strftime('%Y%m%d%H%M'),
           (pd.Timestamp.now()-pd.Timedelta(days=59)).round('30T').strftime('%Y%m%d%H%M')
           ]

# dateRange=[(pd.Timestamp.now()-pd.Timedelta(days=2)).round('30T').strftime('%Y%m%d%H%M'),
#            (pd.Timestamp.now()-pd.Timedelta(days=1)).round('30T').strftime('%Y%m%d%H%M')
#            ]

eP = eddyProAPI.runEP(siteID,
                    userDefinedEddyProSettings=userDefinedEddyProSettings,
                    dateRange=dateRange,
                    processes=3,
                    # debug=True,
                    priority='high',
                    **auxilaryDpaths
                    )

Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_A.eddypro for 17 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_B.eddypro for 16 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_C.eddypro for 16 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_1_fcc.eddypro for 49 files
9
Initiating EddyPro Runs for group 1 on 3 cores at high priority
['c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_A.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_B.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_C.eddypro']
[████████████████████████████████████████████████████████████] 3/3

Remember to update project ID>//////>??????


In [309]:
import configparser
cfg = configparser.ConfigParser()
cfg.read('C:/highfreq//BBS/metadata/group_1_ColDefs.eddypro')
cfg.sections()

['Project']

In [299]:
out = pd.read_csv('C:\\MM_Py\\EddyPro_API\\temp\\eddypro_group_1_fcc_full_output_2024-06-13T171447_adv.csv',
            skiprows=[0,2],header = [0])
# i = 50
# out.columns.get_level_values(0)[i:i+50]
out[['air_pressure','air_p_mean']]#.diff(axis=1)#.dropna().plot()#[out.columns[out.columns.str.contains('co2')]]
# import numpy as np
# raw = pd.read_csv("C:\\highfreq\\BBS\metadata\\rawDataStatistics.csv",
#                   header=[0,1,2],parse_dates=[0],index_col=[0])
# raw.replace(-np.inf, np.nan, inplace=True)
# raw.loc[(raw[('CO2','mg/m3','mean')]-raw[('CO2','mg/m3','mean')].mean()).abs()>300,('CO2','mg/m3','mean')]=np.nan

# # raw[('CO2','mg/m3','mean')].mean()

# raw[('CO2','mg/m3','mean')].plot()

,air_pressure,air_p_mean
0,101256.0,102006.0
1,101256.0,102015.0
2,101256.0,102002.0
3,101256.0,102004.0
4,101256.0,102032.0
5,101256.0,102067.0
6,101256.0,102092.0
7,101256.0,102121.0
8,101256.0,102147.0
9,101256.0,102157.0


In [ ]:
import eddyProAPI
import pandas as pd
import importlib
import time
importlib.reload(eddyProAPI)

# Basic call for .ghg files

siteID = 'BBS'
dateRange=[(pd.Timestamp.now()-pd.Timedelta(days=365)).strftime('%Y%m%d'),pd.Timestamp.now().strftime('%Y%m%d')],


if siteID == 'BB':
    pp = eddyProAPI.preProcessing('BB',dateRange=dateRange,Testing=5)
    pp.searchRawDir()
elif siteID == 'BBS':
                
    # # Basic call for .dat files
    metaDataTemplate='C:/highfreq/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata'
    pp = eddyProAPI.preProcessing('BBS',dateRange=dateRange,fileType='dat',metaDataTemplate=metaDataTemplate,
                                # debug=True,
                                reset=True,
                                # testSet=15
    )
    pp.searchRawDir(
                copyFrom = "C:/Campbellsci/CardConvert",
                searchTag = "BBS.FLUX",
                timeShift=30)
pp.readFiles()
pp.groupAndFilter()
# with shutil 186.0170259475708 seconds
# via subprocess
pp.configurationGroups['Custom']

SystemExit: Quitting

c:\MM_Py\EddyPro_API\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import pandas as pd
kwargs = {'parse_dates': [0],
    'index_col': [0],
    'header': [0,1]}
df = pd.read_csv("Y:\\BB\\metadata\\metaDataValues.csv",**kwargs)

,TIMESTAMP_START,TIMESTAMP_END,DOY_START,DOY_END,FILENAME_HF,SW_IN_POT,NIGHT,EXPECT_NR,FILE_NR,CUSTOM_FILTER_NR,...,CUSTOM_H2O_MEAN,CUSTOM_AIR_P_MEAN,CUSTOM_DIAG_75_MEAN,NUM_BIOMET_VARS,LW_IN_1_1_1,PA_1_1_1,PPFD_IN_1_1_1,RH_1_1_1,SW_IN_1_1_1,TA_1_1_1
"(datetime, )",,,,,,,,,,,,,,,,,,,,,
2024-04-14 12:00:00,202404141200,202404141230,105.4999,105.5207,group_1_TOA5_BBS.FLUX_2024_04_14_1200.dat,1050.42000,0,18000,18000,18000,...,492.365,10205.5,248.000,6,-9999,101.530,-9999,70.2759,-9999,-259.975
2024-04-14 12:30:00,202404141230,202404141300,105.5207,105.5415,group_1_TOA5_BBS.FLUX_2024_04_14_1230.dat,1042.89000,0,18000,18000,18000,...,494.214,10204.7,248.000,6,-9999,101.530,-9999,68.6280,-9999,-259.774
2024-04-14 13:00:00,202404141300,202404141330,105.5415,105.5624,group_1_TOA5_BBS.FLUX_2024_04_14_1300.dat,1020.42000,0,18000,18000,18000,...,485.374,10205.2,248.000,6,-9999,101.530,-9999,67.3566,-9999,-259.370
2024-04-14 13:30:00,202404141330,202404141400,105.5624,105.5832,group_1_TOA5_BBS.FLUX_2024_04_14_1330.dat,983.41600,0,18000,18000,18000,...,483.156,10203.7,248.000,6,-9999,101.520,-9999,64.7052,-9999,-258.680
2024-04-14 14:00:00,202404141400,202404141430,105.5832,105.6040,group_1_TOA5_BBS.FLUX_2024_04_14_1400.dat,932.49400,0,18000,18000,18000,...,467.418,10202.4,248.000,6,-9999,101.510,-9999,67.0983,-9999,-257.883
2024-04-14 14:30:00,202404141430,202404141500,105.6040,105.6249,group_1_TOA5_BBS.FLUX_2024_04_14_1430.dat,868.52900,0,18000,18000,18000,...,494.581,10201.3,248.000,6,-9999,101.495,-9999,63.2546,-9999,-256.221
2024-04-14 15:00:00,202404141500,202404141530,105.6249,105.6457,group_1_TOA5_BBS.FLUX_2024_04_14_1500.dat,792.61500,0,18000,18000,18000,...,470.325,10199.8,248.000,6,-9999,101.480,-9999,62.8824,-9999,-257.233
2024-04-14 15:30:00,202404141530,202404141600,105.6457,105.6665,group_1_TOA5_BBS.FLUX_2024_04_14_1530.dat,706.05300,0,18000,18000,18000,...,467.899,10199.0,248.000,6,-9999,101.485,-9999,62.8328,-9999,-258.171
2024-04-14 16:00:00,202404141600,202404141630,105.6665,105.6873,group_1_TOA5_BBS.FLUX_2024_04_14_1600.dat,610.32300,0,18000,18000,18000,...,459.615,10200.5,248.000,6,-9999,101.490,-9999,64.7296,-9999,-259.006


In [ ]:
## To DO: Add Output ID on merge so we can trace down specific thread/log file for a given run
## Same timestamp can have different outputs depending on batch it was run in
## Usually minor differences, but can be larger in select circumstances

import setupEP as eP
import importlib
import time
importlib.reload(eP)

########## Note - incomplete ghg or biomet files can crash program (preprocessing procedures should help prevent that by re-naming incomplete files)

T1 = time.time()
mR = eP.makeRun('ep_Templates/DefaultSettings.eddypro','BBS',Processes=6,priority = 'high priority')
mR.runDates(['2023-01-01 00:00','2024-03-31 23:59'])

# mR = eP.makeRun('ep_Templates/LabStandard_Advanced.eddypro','BB',Processes=4,priority = 'high priority')
# mR.updateTemplate({'FluxCorrection_SpectralAnalysis_General':
#                     {'sa_mode':4}
#                 })
# mR.runDates(['2015-01-01 00:00','2023-12-31 23:59'])

T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60)

## Read .ghg metadata

In [ ]:
import preProcessing
import importlib
import time
importlib.reload(preProcessing)
T1 = time.time()

# Need one or more metadata templates if working with .dat files
template = ['Y:/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata']

copyFrom = "X:/BBS/EC_Station/2023/20230616"
copyTag = "BBS.FLUX"
pr = preProcessing.read_ALL('BBS',
                            fileType='dat',
                            reset=1,
                            metadataTemplate=template,
                            copyFrom=copyFrom,
                            copyTag=copyTag,
                            timeShift=30)
# pr.find_files(2023,6,processes=4)

# updates = 'Y:/BB/Metadata_Updates.csv'
# pr = preProcessing.read_ALL('BB',fileType='ghg',metadataUpdates=updates,reset=1)
# # If updates/corrections must be made to metadata (e.g., undocumented orientation changes) a .csv files of updates can be provided

# for y in range(2015,2024):
#     for m in range(1,13):
#         pr.find_files(y,m)

# T2 = time.time()

# print('\n')
# print('Runtime: ',(T2-T1)/60,' minutes')


In [ ]:
# ## Implement pre-processing procedures to exclude data by conditions (e.g., low flow rate)
pr.ini['dat']
# import numpy as np
# pr.dataRecords['flow_rate_7200']
# pr.dataRecords.loc[pr.dataRecords['flow_rate_7200'].abs()>1e2,'flow_rate_7200']=np.nan
# pr.dataRecords['flow_rate_7200'].plot()
# # pr.dataRecords['col_air_t'].plot()#['altitude'].describe()
# 317/60

In [ ]:
# https://www.licor.com/env/support/EddyPro/topics/low-pass-filtering.html

# bin_sp_avail=[0,1]
# full_sp_avail=[0,1]
# sa_bin_spectra='Path'
# sa_full_spectra = 'Path'

## Full Runtime

### Reading & Writing all to Y: drive:

1 Month of BB data (20 hz)

**Preprocessing**: 3.91 minutes (8 cores)
**Processing**: 56.92 minutes (6 cores)

Total: **60.83** minutes

### Reading & Preprocessing on Y: drive then writing EddyPro results locally:

1 Month of BB data (20 hz)

**Preprocessing**: 3.91 minutes (8 cores)
**Processing**: 55.74 minutes (6 cores)

Total: **59.65** minutes

### Copying data to C then writing EddyPro results locally:

1 Month of BB data (20 hz)

**Preprocessing**: 6.30 minutes (1 core copy > 8 core preprocessing)

**Processing**: 54.33 minutes (6 cores)

Total: **60.3** minutes

* Paralellizing data copy could shave enought time to get marginal benfit, but doesn't look to matter much

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.read_csv(mR.all_outputs['fulloutput'],skiprows=[0,2],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')
# Add routine to handle duplicates (or find rood cause of them )
# Occuring on boundary between batches I assume as there are ~ 2x as many duplicates as batches
df = df.loc[df.index.duplicated()==False].copy()

bm = pd.read_csv(mR.all_outputs['biomet'],skiprows=[1],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')
bm = bm.loc[bm.index.duplicated()==False].copy()


df = pd.concat([df,bm[bm.columns[~bm.columns.isin(df.columns)]]],axis=1)
df.loc[df['qc_co2_flux']>0,'co2_flux']=np.nan
df.loc[df['u*']<0.15,'co2_flux']=np.nan

plt.scatter(df['Rg_1_1_1'],df['co2_flux'])

# df.loc[df['co2_flux']>20,['wind_speed','u*','wind_dir','Ta_1_1_1','H']]


In [ ]:
import csvFromBinary as cfb
import importlib
importlib.reload(cfb)


# BM
cfb.makeBiomet('BBS',['2023-06-01 00:00','2024-05-31 23:59'],'C:/highfreq/BBS/metadata',type='dynamicMetadata')


In [ ]:
# t = ''
# type(t)

In [ ]:
# df.rename(columns=lambda c: d[c].pop(0) if c in d.keys() else c)
# bm.duplicated().sum()
# bm.columns[bm.columns.duplicated()]
# T1 = df.iloc[0:10].copy()
# T2 = bm[bm.columns[~bm.columns.isin(df.columns)]].iloc[0:10].copy()


# T3 = pd.concat([T1,T2],axis=1)
# T3
rep = df[df.index.duplicated(keep=False)].sort_index().copy()#.duplicated()#['co2_flux']
# print(rep.shape)
for c in rep.columns:
    s = rep.loc[rep[c].duplicated(keep=False)==False,c]

    if s.shape[0]>1:
        # print(c)
        # print(s.shape[0])
        print(rep[['filename',c]].groupby('filename').diff().dropna().min())
        print()

In [ ]:
rep.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
monc = 'Y:/BB/Spectral_Comp/Moncrieff/'+'eddypro_BB_20230601_20230630_full_output_2024-03-15T153700_adv.csv'
frat = 'Y:/BB/Spectral_Comp/Fratini/'+'eddypro_Spectral_Correction_fratini_full_output_2024-03-19T000447_adv.csv'
df_fratini = pd.read_csv(frat,skiprows=[0,2],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')
df_moncrieff = pd.read_csv(monc,skiprows=[0,2],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')

def Filt(df):
    df['filt'] = 1
    df.loc[df['u*']<.15,'filt']=np.nan
    df.loc[df['qc_co2_flux']>0,'filt']=np.nan
    df.loc[df['qc_LE']>0,'filt']=np.nan
    return(df)

df_fratini = Filt(df_fratini)
df_moncrieff = Filt(df_moncrieff)


# df_fratini=df_fratini.loc[df_fratini.index.month==6].copy()

cols = ['filt','u*','co2_flux','qc_co2_flux','LE','qc_LE','H','qc_H','ch4_flux','qc_ch4_flux']
frat_cols = {key: key+'_fratini' for key in cols}
monc_cols = {key: key+'_moncrieff' for key in cols}

df_fratini = df_fratini[cols].rename(columns=frat_cols)
df_moncrieff = df_moncrieff[cols].rename(columns=monc_cols)

df = df_fratini.join(df_moncrieff)

df['filt']=df['filt_fratini']*df['filt_moncrieff']

for col in df.columns:
    if col !='filt':
        df[col]=df[col]*df['filt']


fig,ax=plt.subplots(2,2,figsize=(8,8))


ax[0,0].scatter(df['co2_flux_fratini'],df['co2_flux_moncrieff'],label=f"r2 = {(df[['co2_flux_fratini','co2_flux_moncrieff']].corr()**2)['co2_flux_moncrieff'].round(3).values[0]}")
ax[0,0].set_xlabel('Fratini et al. (2012)')
ax[0,0].set_ylabel('Moncrieff et al. (1997)')
ax[0,0].set_title('CO2 Flux (umol m-2 s-1)')
ax[0,0].legend()
ax[0,0].grid()

df[['co2_flux_fratini','co2_flux_moncrieff']].boxplot(ax=ax[0,1])
ax[0,1].scatter([1,2],df[['co2_flux_fratini','co2_flux_moncrieff']].mean().values,marker='*',s=40,color='k')
ax[0,1].set_xticklabels(['Fratini et al. (2012)','Moncrieff et al. (1997)'])
ax[0,1].set_title('CO2 Flux (umol m-2 s-1)')


ax[1,0].scatter(df['LE_fratini'],df['LE_moncrieff'],label=f"r2 = {(df[['LE_fratini','LE_moncrieff']].corr()**2)['LE_moncrieff'].round(3).values[0]}")
ax[1,0].set_xlabel('Fratini et al. (2012)')
ax[1,0].set_ylabel('Moncrieff et al. (1997)')
ax[1,0].set_title('Latent Heat Flux (W m-2)')
ax[1,0].legend()
ax[1,0].grid()

df[['LE_fratini','LE_moncrieff']].boxplot(ax=ax[1,1])
ax[1,1].scatter([1,2],df[['LE_fratini','LE_moncrieff']].mean().values,marker='*',s=40,color='k')
ax[1,1].set_xticklabels(['Fratini et al. (2012)','Moncrieff et al. (1997)'])
ax[1,1].set_title('Latent Heat Flux (W m-2)')

plt.tight_layout()


df[['LE_fratini','LE_moncrieff','co2_flux_fratini','co2_flux_moncrieff','ch4_flux_fratini','ch4_flux_moncrieff',]].describe().round(3)

In [14]:
import pandas as pd
df = pd.read_csv("G:\\My Drive\\Misc_Stuff\\FishIsland_Processing\\FI_Data_Filled.csv",parse_dates=['datetime'],index_col='datetime')

df.head()
n = 150
df.columns[n:n+50]
df = df.rename(columns={'Table_1':'Water_Table_Depth_1',
                        'Table_2':'Water_Table_Depth_2'})
# df['Canopy_Height_1']
df['Canopy_Height_interp'] = (df['Canopy_Height_1']*0.8+df['Canopy_Height_2']*0.2)*.01

sel = ['AirTC_Avg', 'AirTC_Max', 'AirTC_Min', 'AirTC_Std', 'RH_Max', 'RH_Min',
       'RH_Samp', 'Rain_mm_Tot', 'NR_Wm2_Avg', 'NR_Wm2_Max', 'NR_Wm2_Min',
       'NR_Wm2_Std', 'PPFD_Avg', 'PPFD_Max', 'PPFD_Min', 'PPFD_Std',
       'Temp_2_5_1', 'Temp_5_1', 'Temp_15_1', 'Temp_2_5_2', 'Temp_5_2', 'Temp_15_2', 'Active_Layer_1', 'Canopy_Height_1', 'Water_Table_Depth_1',
        'Active_Layer_2', 'Canopy_Height_2', 'Water_Table_Depth_2','Canopy_Height_interp']
# df[['Canopy_Height_est','Zo']].plot()
df = df.loc[df.index>='2017-06-23']
df[sel].to_csv('G:\\My Drive\\Misc_Stuff\\FishIsland_2017\\HalfHourlyData.csv')
df.head()

,UTC,DOY,daytime,file_records,used_records,Tau,qc_Tau,H,qc_H,LE,...,ch4_flux_drop,Rain_diff,ch4_noSSFilter,PBLH,HR,TempFill,co2_flux_Filled,ch4_flux_Filled,Canopy_Height_est,Canopy_Height_interp
datetime,,,,,,,,,,,,,,,,,,,,,
2017-06-23 00:00:00,2017-06-23 00:00:00,174.0,0.0,1800.0,1800.0,0.034567,0.0,7.066999,1.0,5.624018,...,0,0.0,0.014609,1025.800049,0.0,0.026088,-0.102759,0.014609,NaN,NaN
2017-06-23 00:30:00,2017-06-23 00:30:00,174.0,0.0,1800.0,1800.0,0.034080,0.0,-18.442653,0.0,NaN,...,0,0.0,0.008549,959.901711,0.0,0.027147,0.180692,0.008549,NaN,NaN
2017-06-23 01:00:00,2017-06-23 01:00:00,174.0,0.0,1800.0,1800.0,0.050775,1.0,-12.098934,0.0,NaN,...,0,0.0,0.010209,894.003372,1.0,0.027080,0.140026,0.010209,NaN,NaN
2017-06-23 01:30:00,2017-06-23 01:30:00,174.0,0.0,1800.0,1800.0,0.039778,1.0,-10.078745,0.0,NaN,...,0,0.0,0.010809,828.105034,1.0,0.026860,0.201040,0.010809,NaN,NaN
2017-06-23 02:00:00,2017-06-23 02:00:00,174.0,0.0,1800.0,1800.0,0.032883,1.0,-8.495377,0.0,0.750913,...,0,0.0,0.011372,762.206696,2.0,0.023808,0.137105,0.011372,NaN,NaN


# Speedtests

All times in minutes

### 4 threads (8 timesteps each)

Normal priority: 2.05
High priority: 1.91

### 4 threads (48 timesteps each x 12 days)

High priority: 29

### Eddypro GUI (1 run [12 days, 576 timesteps])

High priority: 90